In [2]:
!pip list | grep qiskit

qiskit                        0.46.1
qiskit-aer                    0.13.3
qiskit-ibm-runtime            0.20.0
qiskit-terra                  0.46.1


In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService, Options, Sampler, Session, Estimator
from qiskit import QuantumCircuit

from qiskit_aer import AerSimulator


from qiskit import IBMQ, Aer

from qiskit_ibm_runtime.fake_provider import FakeManilaV2

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit.quantum_info import SparsePauliOp

from torchquantum.measurement import expval_joint_analytical

from torchquantum.measurement import expval_joint_sampling, expval_joint_sampling_grouping

In [2]:
import math
import pdb
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import torchquantum as tq
import qiskit_aer


In [3]:
import torchquantum.functional as tqf
import argparse
import tqdm
import time

import torch
import torch.nn.functional as F
#import torchtext.legacy
#from torchtext.legacy import data, datasets, vocab

In [15]:
from torchquantum.plugin.qiskit import tq2qiskit

In [4]:
from dotenv import dotenv_values

config = dotenv_values(".env")

In [5]:
service = QiskitRuntimeService(channel="ibm_quantum", token=config["IBM_TOKEN"])

In [6]:
options = Options(optimization_level=1, execution={"shots":32})

In [7]:
service.backends()

[<IBMBackend('ibm_brisbane')>,
 <IBMBackend('ibm_kyoto')>,
 <IBMBackend('ibm_sherbrooke')>,
 <IBMBackend('ibmq_qasm_simulator')>,
 <IBMBackend('simulator_extended_stabilizer')>,
 <IBMBackend('simulator_stabilizer')>,
 <IBMBackend('ibm_osaka')>,
 <IBMBackend('simulator_mps')>,
 <IBMBackend('simulator_statevector')>]

In [8]:
backend = service.backend("ibmq_qasm_simulator")

----

Little test backend service and Estimator :

In [39]:
from qiskit import QuantumCircuit
 
qc = QuantumCircuit(2, 2)
qc.h(0)
#qc.h(1)
#qc.h(0)
#qc.h(1)
#qc.x(0)
qc.x(1)
#qc.cnot(0,1)
qc.draw()

┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ X ├
     └───┘
c: 2/═════

In [40]:
qdev = tq.QuantumDevice(n_wires=2, bsz=1, device="cpu") # use device='cuda' for GPU


In [81]:
class TQTest(tq.QuantumModule):
        def __init__(self, n_qubits, *args, **kwargs):
            super().__init__()
            self.n_wires = n_qubits
            #self.rx_0 = tq.RX(has_params=True, trainable=True)
        @tq.static_support
        def forward(self, q_device, return_q_dev=False):
            tqf.h(q_device, wires=0, static=self.static_mode, parent_graph=self.graph) 
            tqf.x(q_device, wires=1, static=self.static_mode, parent_graph=self.graph)
            #self.rx_0(q_device, wires=0)
             
            q_device = q_device.bfloat16()

            if return_q_dev:
                 return q_device


In [84]:
tqtest = TQTest(n_qubits=2)

In [83]:
for param in tqtest.parameters():
    print(param)

In [85]:
observables = ["ZI", "IZ"]
expval_joint_sampling_grouping(qdev=tqtest(qdev, return_q_dev=True), observables=observables, n_shots_per_group=32)

{'ZI': tensor([0.]), 'IZ': tensor([-1.])}

In [66]:
qc_2 = tq2qiskit(q_device=qdev, m=tqtest)

In [67]:
qc.draw()

┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ X ├
     └───┘
c: 2/═════

In [68]:
qc_2.draw()

┌───┐┌────────────┐
q_0: ┤ H ├┤ Rx(1.4349) ├
     ├───┤└────────────┘
q_1: ┤ X ├──────────────
     └───┘

In [ ]:
observables = ["ZI", "IZ"]


with Session(service=service, backend=backend) as session:

    estimator = Estimator(session=session, options=options)
    job = estimator.run(circuits=[qc, qc], observables=observables)
    print(f"Estimator results: {job.result()}")

In [60]:
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
circ_test = pm.run(qc)

In [61]:
circ_test.draw()

global phase: π/4
     ┌─────────┐┌────┐┌─────────┐
q_0: ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├
     └──┬───┬──┘└────┘└─────────┘
q_1: ───┤ X ├────────────────────
        └───┘                    
c: 2/════════════════════════════

In [62]:
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
circ_test_2 = pm.run(qc_2)

circ_test_2.draw()

global phase: π/4
     ┌─────────┐┌────┐┌─────────┐
q_0: ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├
     └──┬───┬──┘└────┘└─────────┘
q_1: ───┤ X ├────────────────────
        └───┘

In [11]:
observables = ["ZI", "IZ"]


with Session(service=service, backend=backend) as session:

    estimator = Estimator(session=session, options=options)
    job = estimator.run(circuits=[circ_test, circ_test], observables=observables)
    print(f"Estimator results: {job.result()}")

/tmp/ipykernel_11621/457931810.py:7: DeprecationWarning: Circuits that do not match the target hardware definition will no longer be supported after March 1, 2024. See the transpilation documentation (https://docs.quantum.ibm.com/transpile) for instructions to transform circuits and the primitive examples (https://docs.quantum.ibm.com/run/primitives-examples) to see this coupled with operator transformations.
  job = estimator.run(circuits=[circ_test, circ_test], observables=observables)
/home/jesshuan/miniconda3/envs/torch_quantum/lib/python3.12/site-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:935: UserWarning: Cloud simulators have been deprecated and will be removed on 15 May 2024. Use the new local testing mode in qiskit-ibm-runtime version 0.22.0 or later to meet your debugging needs.
  warnings.warn(warning_message)


Estimator results: EstimatorResult(values=array([-1.,  0.]), metadata=[{'variance': 0.0, 'shots': 32}, {'variance': 1.0, 'shots': 32}])


In [63]:
observables = ["ZI", "IZ"]


with Session(service=service, backend=backend) as session:

    estimator = Estimator(session=session, options=options)
    job = estimator.run(circuits=[circ_test_2, circ_test_2], observables=observables)
    print(f"Estimator results: {job.result()}")

/tmp/ipykernel_11621/1729889922.py:7: DeprecationWarning: Circuits that do not match the target hardware definition will no longer be supported after March 1, 2024. See the transpilation documentation (https://docs.quantum.ibm.com/transpile) for instructions to transform circuits and the primitive examples (https://docs.quantum.ibm.com/run/primitives-examples) to see this coupled with operator transformations.
  job = estimator.run(circuits=[circ_test_2, circ_test_2], observables=observables)
/home/jesshuan/miniconda3/envs/torch_quantum/lib/python3.12/site-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:935: UserWarning: Cloud simulators have been deprecated and will be removed on 15 May 2024. Use the new local testing mode in qiskit-ibm-runtime version 0.22.0 or later to meet your debugging needs.
  warnings.warn(warning_message)


Estimator results: EstimatorResult(values=array([-1.,  0.]), metadata=[{'variance': 0.0, 'shots': 32}, {'variance': 1.0, 'shots': 32}])


-----

In [86]:
class MultiHeadAttentionBase(nn.Module):
    def __init__(self,
                 embed_dim: int,
                 num_heads: int,
                 dropout: float = 0.1,
                 mask=None,
                 use_bias=False):
        super(MultiHeadAttentionBase, self).__init__()

        assert embed_dim % num_heads == 0, f"Embedding dimension ({embed_dim}) should be divisible by number of heads ({num_heads})"

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.d_k = embed_dim // num_heads  # projection dimensions
        self.k_linear = None
        self.q_linear = None
        self.v_linear = None
        self.combine_heads = None
        self.dropout = nn.Dropout(dropout)
        self.attn_weights = None
    
    def separate_heads(self, x):
        '''
        split into N heads
        from (batch_size, seq_len, embed_dim)
        to   (batch_size, seq_len, num_heads, embed_dim)
        then transpose (1,2) to (batch_size, num_heads, seq_len, embed_dim)
        to make mat mult straightforward for each head
        '''
        batch_size = x.size(0)
        x = x.view(batch_size, -1, self.num_heads, self.d_k)
        return x.transpose(1, 2)

    def attention(self, query, key, value, mask=None, dropout=None):
        '''
        Attention(Q, K, V) = softmax(Q K^T / sqrt(d_k))V
        '''
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(self.d_k)
        # see also: https://tensorchiefs.github.io/dlday2018/tutorial/einsum.html
        #scores = torch.einsum('bijh, bkjh -> bikh', query, key) / math.sqrt(self.d_k)
        if mask is not None:
            mask = mask.unsqueeze(1)
            scores = scores.masked_fill(mask == 0, -1e9)
        scores = F.softmax(scores, dim=-1)
        if dropout is not None:
            scores = dropout(scores)
        attn = torch.matmul(scores, value)
        return attn, scores
    
    def downstream(self, query, key, value, batch_size, mask=None):
        Q = self.separate_heads(query)
        K = self.separate_heads(key)
        V = self.separate_heads(value)

        x, self.attn_weights = self.attention(Q, K, V, mask, dropout=self.dropout)

        concat = x.transpose(1, 2).contiguous().view(batch_size, -1, self.embed_dim)

        return concat
        # output = self.combine_heads(concat)
        # return output

   # def forward(self, x, mask=None):
    #    raise NotImplementedError("Base class does not execute forward function.")
        
        


In [87]:
class MultiHeadAttentionClassical(MultiHeadAttentionBase):
    
    def __init__(self, embed_dim: int,
                 num_heads: int,
                 dropout=0.1,
                 mask=None,
                 use_bias=False):
        super(MultiHeadAttentionClassical, self).__init__(embed_dim=embed_dim, num_heads=num_heads, dropout=dropout, mask=mask, use_bias=use_bias)

        self.k_linear = nn.Linear(embed_dim, embed_dim, bias=use_bias)
        self.q_linear = nn.Linear(embed_dim, embed_dim, bias=use_bias)
        self.v_linear = nn.Linear(embed_dim, embed_dim, bias=use_bias)
        self.combine_heads = nn.Linear(embed_dim, embed_dim, bias=use_bias)
    
    def forward(self, x, mask=None):
        batch_size, seq_len, embed_dim = x.size()
        assert embed_dim == self.embed_dim, f"Input embedding ({embed_dim}) does not match layer embedding size ({self.embed_dim})"

        K = self.k_linear(x)
        Q = self.q_linear(x)
        V = self.v_linear(x)

        x = self.downstream(Q, K, V, batch_size, mask)
        output = self.combine_heads(x)
        return output

In [88]:
class QLayer(tq.QuantumModule):
        def __init__(self, n_qbits, *args, **kwargs):
            super().__init__()    
            self.n_wires = n_qbits
            self.encoder = tq.GeneralEncoder(
                    [{'input_idx': [i], 'func': 'rx', 'wires': [i]} for i in range(self.n_wires)])
            #self.rx_list = [tq.RX(has_params=True, trainable=True) for _ in range(self.n_wires)]
            #self.ry_test = tq.RY(has_params=True, trainable=True)
            #self.measure = tq.MeasureAll(tq.PauliZ)
            if n_qbits >= 2:
                self.rx_0 = tq.RX(has_params=True, trainable=True)
                self.rx_1 = tq.RX(has_params=True, trainable=True)
            if n_qbits >= 4:
                self.rx_2 = tq.RX(has_params=True, trainable=True)
                self.rx_3 = tq.RX(has_params=True, trainable=True)
            if n_qbits >= 8:
                self.rx_4 = tq.RX(has_params=True, trainable=True)
                self.rx_5 = tq.RX(has_params=True, trainable=True)
                self.rx_6 = tq.RX(has_params=True, trainable=True)
                self.rx_7 = tq.RX(has_params=True, trainable=True)
            if n_qbits >= 16:
                self.rx_8 = tq.RX(has_params=True, trainable=True)
                self.rx_9 = tq.RX(has_params=True, trainable=True)
                self.rx_10 = tq.RX(has_params=True, trainable=True)
                self.rx_11 = tq.RX(has_params=True, trainable=True)
                self.rx_12 = tq.RX(has_params=True, trainable=True)
                self.rx_13 = tq.RX(has_params=True, trainable=True)
                self.rx_14 = tq.RX(has_params=True, trainable=True)
                self.rx_15 = tq.RX(has_params=True, trainable=True)
            if n_qbits >= 32:
                self.rx_16 = tq.RX(has_params=True, trainable=True)
                self.rx_17 = tq.RX(has_params=True, trainable=True)
                self.rx_18 = tq.RX(has_params=True, trainable=True)
                self.rx_19 = tq.RX(has_params=True, trainable=True)
                self.rx_20 = tq.RX(has_params=True, trainable=True)
                self.rx_21 = tq.RX(has_params=True, trainable=True)
                self.rx_22 = tq.RX(has_params=True, trainable=True)
                self.rx_23 = tq.RX(has_params=True, trainable=True)
                self.rx_24 = tq.RX(has_params=True, trainable=True)
                self.rx_25 = tq.RX(has_params=True, trainable=True)
                self.rx_26 = tq.RX(has_params=True, trainable=True)
                self.rx_27 = tq.RX(has_params=True, trainable=True)
                self.rx_28 = tq.RX(has_params=True, trainable=True)
                self.rx_29 = tq.RX(has_params=True, trainable=True)
                self.rx_30 = tq.RX(has_params=True, trainable=True)
                self.rx_31 = tq.RX(has_params=True, trainable=True)

            #self.observables = SparsePauliOp.from_list([("ZX" + "I"*(n_qbits - 2),1), ("XY" + "I"*(n_qbits - 2),1)] + [("I"*i + "Z" + "I"*(n_qbits - 1 -i), 1) for i in range(n_qbits)])
            #self.measure = tq.MeasureAll(tq.PauliZ)
            #self.measure = tq.MeasureMultiPauliSum(self.observables)

        def ansatz_gate_forward(self, q_device):
            if self.n_wires >= 2:
                self.rx_0(q_device, wires=0)
                self.rx_1(q_device, wires=1)
            if self.n_wires >= 4:
                self.rx_2(q_device, wires=2)
                self.rx_3(q_device, wires=3)
            if self.n_wires >= 8:
                self.rx_4(q_device, wires=4)
                self.rx_5(q_device, wires=5)
                self.rx_6(q_device, wires=6)
                self.rx_7(q_device, wires=7)
            if self.n_wires >= 16:
                self.rx_8(q_device, wires=8)
                self.rx_9(q_device, wires=9)
                self.rx_10(q_device, wires=10)
                self.rx_11(q_device, wires=11)
                self.rx_12(q_device, wires=12)
                self.rx_13(q_device, wires=13)
                self.rx_14(q_device, wires=14)
                self.rx_15(q_device, wires=15)
            if self.n_wires >= 32:
                self.rx_16(q_device, wires=16)
                self.rx_17(q_device, wires=17)
                self.rx_18(q_device, wires=18)
                self.rx_19(q_device, wires=19)
                self.rx_20(q_device, wires=20)
                self.rx_21(q_device, wires=21)
                self.rx_22(q_device, wires=22)
                self.rx_23(q_device, wires=23)
                self.rx_24(q_device, wires=24)
                self.rx_25(q_device, wires=25)
                self.rx_26(q_device, wires=26)
                self.rx_27(q_device, wires=27)
                self.rx_28(q_device, wires=26)
                self.rx_29(q_device, wires=29)
                self.rx_30(q_device, wires=30)
                self.rx_31(q_device, wires=31)
        
        @tq.static_support
        def forward(self, q_device, x, return_q_device=False):
            self.encoder(q_device, x)
            #for k in range(self.n_wires):
                 #self.rx_list[k](q_device, wires=k)
            #self.ry_test(q_device, wires=0)
            self.ansatz_gate_forward(q_device)

            for k in range(self.n_wires):
                if k==self.n_wires-1:
                    tqf.cnot(q_device, wires=[k, 0], static=self.static_mode, parent_graph=self.graph) 
                else:
                    tqf.cnot(q_device, wires=[k, k+1], static=self.static_mode, parent_graph=self.graph)
            q_device = q_device.bfloat16()
            
            if return_q_device:
                return q_device


In [125]:
obs = ["I"*i + "ZX"+ "I"*(4 - 2 - i) for i in range(4 - 1)] + ["XX" + "I"*(4-2)]

In [132]:
obs

['ZXII', 'IZXI', 'IIZX', 'XXII']

In [133]:
["".join(reversed(obss)) for obss in obs]

['IIXZ', 'IXZI', 'XZII', 'IIXX']

In [113]:
from copy import copy

In [153]:
class MultiHeadAttentionQuantum(MultiHeadAttentionBase):
    
            
    def __init__(self,
                 embed_dim: int,
                 num_heads: int,
                 dropout=0.1,
                 mask=None,
                 use_bias=False,
                 n_qubits: int = 4,
                 n_qlayers: int = 1,
                 q_device="default.qubit",
                 session = None):
        super(MultiHeadAttentionQuantum, self).__init__(embed_dim, num_heads, dropout=dropout, mask=mask, use_bias=use_bias)
        
        # todo: add intermediate layer to "dress" quantum circuit
        assert n_qubits == embed_dim, "Number of qubits ({n_qubits}) does not match embedding dim ({embed_dim})"
        self.n_qubits = n_qubits
        #self.n_qlayers = n_qlayers

        self.k_observables = ["Z"*(n_qubits - 1)]
        self.q_observables = ["Z"*(n_qubits - 1)]
        self.v_observables = ["I"*i + "ZX"+ "I"*(n_qubits - 2 - i) for i in range(n_qubits - 1)] + ["XX" + "I"*(n_qubits - 2)]

        self.v_observables_reversed = ["".join(reversed(obs)) for obs in self.v_observables]

        self.k_layer = QLayer(n_qubits)
        self.q_layer = QLayer(n_qubits)
        self.v_layer = QLayer(n_qubits)
        #self.measure = tq.MeasureAll(tq.PauliZ)
        self.q_device = q_device
        self.session = session

    def forward(self, x, mask=None):
        batch_size, seq_len, embed_dim = x.size()
        assert embed_dim == self.embed_dim, f"Input embedding ({embed_dim}) does not match layer embedding size ({self.embed_dim})"

        q_dev = tq.QuantumDevice(n_wires=self.n_qubits, device=self.q_device, bsz=x.shape[0])

        options = Options(optimization_level=1, execution={})

        if self.session is not None:
            options = Options(optimization_level=1, execution={"shots":1024})
            estimator = Estimator(session=self.session, options=options)
            print(self.v_observables)
            V = [estimator.run(circuits=[tq2qiskit(q_device=q_dev, m=self.v_layer, x=x[:, t, :].clone()) for o in range(len(self.v_observables))],
                                     observables=self.v_observables).result() for t in range(seq_len)]
            print(V)
        else:
            print(self.v_observables)
            print(self.v_observables_reversed)
            print([expval_joint_sampling_grouping(qdev=self.v_layer(q_dev, x[:, t, :].clone(), return_q_device=True), observables=self.v_observables_reversed, n_shots_per_group=1024) for t in range(seq_len)])
            #print(torch.stack([torch.stack(list(expval_joint_sampling_grouping(qdev=self.v_layer(q_dev, x[:, t, :].clone(), return_q_device=True), observables=self.v_observables, n_shots_per_group=1024).values())) for t in range(seq_len)]))
            #V = [self.v_layer(q_dev, x[:, t, :].clone()).values for t in range(seq_len)]
        
        #print(V)
        
        #K = [self.q_layer(x[:, t, :].clone(),q_dev) for t in range(seq_len)]
        #Q = [self.q_layer(x[:, t, :].clone(),q_dev) for t in range(seq_len)]
        #V = [self.q_layer(x[:, t, :].clone(),q_dev) for t in range(seq_len)]

        K = torch.Tensor(pad_sequence(K))
        Q = torch.Tensor(pad_sequence(Q))
        V = torch.Tensor(pad_sequence(V))
        x = self.downstream(Q, K, V, batch_size, mask)
        #output = [self.q_layer(x[:, t, :],q_dev) for t in range(seq_len)]
        #output = torch.Tensor(pad_sequence(output)).clone()
        return x


In [154]:
EMBED_DIM = 4

BATCH_SIZE = 1

SEQ_LEN = 1


In [155]:
classical_module = MultiHeadAttentionClassical(embed_dim=EMBED_DIM, num_heads=4, dropout=0.0)
#quantum_module = MultiHeadAttentionQuantum(embed_dim=EMBED_DIM, num_heads=4, dropout=0.0, n_qubits=EMBED_DIM, q_device="cuda", session=session)

In [156]:
test_input = torch.tensor(np.random.rand(BATCH_SIZE, SEQ_LEN, EMBED_DIM), dtype=torch.float32)

In [157]:
output = classical_module(test_input)

In [158]:
output.shape

torch.Size([1, 1, 4])

In [159]:
quantum_module = MultiHeadAttentionQuantum(embed_dim=EMBED_DIM, num_heads=4, dropout=0.0, n_qubits=EMBED_DIM, q_device="cpu")

In [160]:
output_q = quantum_module(test_input)

['ZXII', 'IZXI', 'IIZX', 'XXII']
['IIXZ', 'IXZI', 'XZII', 'IIXX']
[{'IXZI': tensor([-0.0391]), 'IIXZ': tensor([-0.0664]), 'XZII': tensor([-0.2559]), 'IIXX': tensor([-0.0137])}]


UnboundLocalError: cannot access local variable 'K' where it is not associated with a value

In [151]:
with Session(backend=backend) as session:
    quantum_module = MultiHeadAttentionQuantum(embed_dim=EMBED_DIM, num_heads=4, dropout=0.0, n_qubits=EMBED_DIM, q_device="cpu", session=session)
    output_q = quantum_module(test_input)

['ZXII', 'IZXI', 'IIZX', 'XXII']


/tmp/ipykernel_11621/4187400498.py:46: DeprecationWarning: Circuits that do not match the target hardware definition will no longer be supported after March 1, 2024. See the transpilation documentation (https://docs.quantum.ibm.com/transpile) for instructions to transform circuits and the primitive examples (https://docs.quantum.ibm.com/run/primitives-examples) to see this coupled with operator transformations.
  V = [estimator.run(circuits=[tq2qiskit(q_device=q_dev, m=self.v_layer, x=x[:, t, :].clone()) for o in range(len(self.v_observables))],


[EstimatorResult(values=array([-0.265625  ,  0.41992188, -0.47851562,  0.03125   ]), metadata=[{'variance': 0.929443359375, 'shots': 1024}, {'variance': 0.8236656188964844, 'shots': 1024}, {'variance': 0.7710227966308594, 'shots': 1024}, {'variance': 0.9990234375, 'shots': 1024}])]


UnboundLocalError: cannot access local variable 'K' where it is not associated with a value